In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.16.1


# step1 
    docker pull tensorflow/serving

In [5]:
import tensorflow as tf
## 样本数量
n = 800

## 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1], mean = 0.0,stddev= 2.0) # @表示矩阵乘法,增加正态扰动

## 建立模型
tf.keras.backend.clear_session()
inputs = tf.keras.Input(shape = (2,),name ="inputs") #设置输入名字为inputs
outputs = tf.keras.layers.Dense(1, name = "outputs")(inputs) #设置输出名字为outputs
linear = tf.keras.models.Model(inputs = inputs,outputs = outputs)
## 使用fit方法进行训练
linear.compile(optimizer="rmsprop",loss="mse",metrics=["mae"])
linear.fit(X,Y,batch_size = 8,epochs = 10)  

tf.print("w = ",linear.layers[1].kernel)
tf.print("b = ",linear.layers[1].bias)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 125.5804 - mae: 9.5044
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 113.7262 - mae: 9.0244
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 102.5681 - mae: 8.4743
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 100.6957 - mae: 8.5454
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 83.1050 - mae: 7.7396
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 76.3963 - mae: 7.3340
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 61.7910 - mae: 6.6524
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 57.6300 - mae: 6.3538
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 48.1902 - mae: 5.8354
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 45.1696 - mae: 5.5870
w =  [[1.05603242]
 [-0.911676288]]
b =  [0.676320493]


In [10]:
linear.save("./data/1/linear.keras")

In [11]:
try:
    tf.saved_model.save(linear,"data/tmp")
except Exception as e:
    print(e)

this __dict__ descriptor does not support '_DictWrapper' objects


In [12]:
# 查看模型文件相关信息
!saved_model_cli show --dir './data' --all

2024-04-24 17:24:31.963610: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/Users/yangyw/miniconda3/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/yangyw/miniconda3/lib/python3.12/site-packages/tensorflow/python/tools/saved_model_cli.py", line 1340, in main
    app.run(smcli_main)
  File "/Users/yangyw/miniconda3/lib/python3.12/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/Users/yangyw/miniconda3/lib/python3.12/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
             ^^^^^^^^^^
  File "/Users/yangyw/miniconda3/lib/python3.12/site-packages/tensorflow/python/tools/saved_model_cli.py", line 1338, in

In [25]:
!docker run -t --rm -p 8501:8501 \
    -v "./data/" \
    -e MODEL_NAME=linear_model \
    tensorflow/serving & >server.log 2>&1

2024-04-24 06:53:23.231273: I tensorflow_serving/model_servers/server.cc:77] Building single TensorFlow model file config:  model_name: linear_model model_base_path: /models/linear_model
2024-04-24 06:53:23.237733: I tensorflow_serving/model_servers/server_core.cc:471] Adding/updating models.
2024-04-24 06:53:23.237848: I tensorflow_serving/model_servers/server_core.cc:600]  (Re-)adding model: linear_model
2024-04-24 06:53:23.242477: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:358] FileSystemStoragePathSource encountered a filesystem access error: Could not find base path /models/linear_model for servable linear_model with error NOT_FOUND: /models/linear_model not found
2024-04-24 06:53:24.241976: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:358] FileSystemStoragePathSource encountered a filesystem access error: Could not find base path /models/linear_model for servable linear_model with error NOT_FOUND: /models/linear_mo

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.BackupAndRestore('backup'),
]

model.fit(x_train, y_train, epochs=5, callbacks=callbacks)

/Users/yangyw/miniconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.8589 - loss: 0.4810
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9553 - loss: 0.1514
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9680 - loss: 0.1080
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9744 - loss: 0.0849
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9762 - loss: 0.0739


In [3]:
tf.saved_model.save(model,"data/tmp")

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects